In [1]:
import numpy as np
from math import *
import scipy.special as sp
import matplotlib.pyplot as plt

 ## Minimization of Nice function

In [2]:
def func_ND(parameter):
    x,y,z=parameter
    return 1.5-sp.jv(0,(5*(x-0.5)**2 + (0.5*x+y-0.5)**2 + 0.2*(0.25*x+0.5*y+z-0.5)**2))

### $Powell's$ $Algorithm$

In [3]:
def golden_ratio(function, e, current, a, b, tolerance):
    c = (3 - sqrt(5)) / 2
    x1 = a + (b - a) * c
    x2 = a + b - x1

    fx1 = function(current + x1 * e)
    fx2 = function(current + x2 * e)

    while b - a > tolerance:
        if fx1 <= fx2:
            b = x2
            x2 = x1
            fx2 = fx1
            x1 = a + c * (b - a)
            fx1 = function(current + x1 * e)
            x = x1
        else:
            a = x1
            x1 = x2
            fx1 = fx2
            x2 = b - c * (b - a)
            fx2 = function(current + x2 * e)
            x = x2

    return x

In [4]:
def powell(function, helper_function, starting_position, tolerance, display_info=False, window=None, gda=0, gdb=1):
    dimension = starting_position.size
    e = np.eye(dimension)
    x = starting_position
    x1 = starting_position + 2 * 10
    num_iterations = 0
    minlist=[]

    while np.max(np.abs(x - x1)) > tolerance:
        num_iterations += 1
        current = x
        for i in range(dimension):
            theta = helper_function(function, e[:, i], current, gda, gdb, tolerance)#Calculating the minima 
            current = current + theta * e[:, i]                                     #along the dimension

        for i in range(dimension - 1):# x,y,z--> y,z,v1 and so on, this loop does the droping of direction
            e[:, i] = e[:, i + 1]
        e[:, dimension - 1] = current - x

        x1 = x
        theta = helper_function(function, e[:, dimension - 1], current, gda, gdb, tolerance)
        x = current + theta * e[:, dimension - 1]
        minlist.append(x)

       
    fx = function(x)

    return x, fx, num_iterations,e,np.array(minlist)

In [5]:
b = np.array([0,0,0]).T
ans_powell_nice=powell(func_ND, golden_ratio, b, 1e-8)

In [6]:
ans_powell_nice

(array([0.52731591, 0.26015149, 0.23809492]),
 0.5000046175144227,
 3,
 array([[5.27315901e-01, 1.42422174e-09, 1.42422174e-09],
        [2.60151484e-01, 3.40352935e-09, 7.02640279e-10],
        [2.38094878e-01, 3.32972759e-08, 3.34395703e-09]]),
 array([[0.5273159 , 0.26015148, 0.23809488],
        [0.5273159 , 0.26015149, 0.23809491],
        [0.52731591, 0.26015149, 0.23809492]]))

In [7]:
ans_powell_nice[3]# Conjugate direction

array([[5.27315901e-01, 1.42422174e-09, 1.42422174e-09],
       [2.60151484e-01, 3.40352935e-09, 7.02640279e-10],
       [2.38094878e-01, 3.32972759e-08, 3.34395703e-09]])

In [8]:
def eigenvalues_eigenvector(mat):
    mat = np.array(mat, dtype=float)
    eigen_values,eigen_vector=np.linalg.eig(mat)
    
    sorted_eigen_values=np.sort(eigen_values)[::-1]
    j=np.argsort(eigen_values)[::-1]
    
    sorted_eigen_vector=np.zeros((3,3),dtype='float')
    for i in range(sorted_eigen_values.shape[0]):
        sorted_eigen_vector[:,i]=eigen_vector[:,j[i]]
        
    return sorted_eigen_values,sorted_eigen_vector
        

 ### $Minimizing$ $along$ $Cordinate$ $directions$

In [9]:
def f(x):
    val=(x[0]-0.5)**2+(0.5*x[0]+x[1]-0.5)**2+(0.25*x[0]+0.5*x[1]+x[2]-0.5)**2
    return 1.5-sp.jv(0,val)

In [10]:
pos=np.array([0,0,0],dtype=np.float64)
deltax=np.array([1e-4,0,0])
deltay=np.array([0,1e-4,0])
deltaz=np.array([0,0,1e-4])
dirs=[deltax,deltay,deltaz]
for i in range(10):
    prev=f(pos)
    if f(pos)>f(pos+dirs[0]):
        while f(pos)>f(pos+dirs[0]):
            pos=pos+dirs[0]
    elif(f(pos)>f(pos-dirs[0])):
        while f(pos)>f(pos-dirs[0]):
            pos=pos-dirs[0]
    if f(pos)>f(pos+dirs[1]):
        while f(pos)>f(pos+dirs[1]):
            pos=pos+dirs[1]
    elif f(pos)>f(pos-dirs[1]):
        while f(pos)>f(pos-dirs[1]):
            pos=pos-dirs[1]
    if f(pos)>f(pos+dirs[2]):
        while f(pos)>f(pos+dirs[2]):
            pos=pos+dirs[2]
    elif f(pos)>f(pos-dirs[2]):
        while f(pos)>f(pos-dirs[2]):
            pos=pos-dirs[2]
    print (pos, f(pos))
    if abs(prev-f(pos))<1e-10:
        break

[0.6667 0.2666 0.2   ] 0.5003567793641424
[0.5016 0.2692 0.24  ] 0.500000040513638
[0.4928 0.2576 0.248 ] 0.5000000011505664
[0.4968 0.2524 0.2496] 0.5000000000295935
[0.4989 0.2507 0.2499] 0.5000000000003801


### $Conjugate$ $gradient$

In [11]:
def step_length(f, g, xk, alpha, pk, c2):
    return interpolation(f, g,
                       lambda alpha: f(xk + alpha * pk),
                       lambda alpha: np.dot(g(xk + alpha * pk), pk),
                       alpha, c2,
                       lambda f, g, alpha, c2: strong_wolfe(f, g, xk, alpha, pk, c2))

In [12]:
def wolfe(f, g, xk, alpha, pk):
    c1 = 1e-4
    return f(xk + alpha * pk) <= f(xk) + c1 * alpha * np.dot(g(xk), pk)


def strong_wolfe(f, g, xk, alpha, pk, c2):
  # typically, c2 = 0.9 when using Newton or quasi-Newton's method.
  #            c2 = 0.1 when using non-linear conjugate gradient method.
      return wolfe(f, g, xk, alpha, pk) and abs(
           np.dot(g(xk + alpha * pk), pk)) <= c2 * abs(np.dot(g(xk), pk))

In [13]:
def interpolation(f, g, f_alpha, g_alpha, alpha, c2, strong_wolfe_alpha, iters=20):
  # referred implementation here:
  # https://github.com/tamland/non-linear-optimization
    l = 0.0
    h = 1.0
    for i in range(iters):
        if strong_wolfe_alpha(f, g, alpha, c2):
            return alpha

        half = (l + h) / 2
        alpha = - g_alpha(l) * (h**2) / (2 * (f_alpha(h) - f_alpha(l) - g_alpha(l) * h))
        if alpha < l or alpha > h:
            alpha = half
        if g_alpha(alpha) > 0:
            h = alpha
        elif g_alpha(alpha) <= 0:
            l = alpha
    return alpha


In [34]:
def conjugate_gradient(f, g, x0, iterations, error):
    
    xk = x0
    c2 = 0.1

    fk = f(xk)
    gk = g(xk)
    pk = -1*gk
    count=0
    plist=[pk]
    glist=[gk]

    for i in range(iterations):
        alpha = step_length(f, g, xk, 1.0, pk, c2)
        xk1 = xk + alpha * pk
        gk1 = g(xk1)
        beta_k1 = np.dot(gk1, gk1) / np.dot(gk, gk)
        pk1 = -gk1 + beta_k1 * pk #conjugate directions
        count=count+1
        plist.append(pk1)
        glist.append(gk1)
        if np.linalg.norm(xk1 - xk) < error:
            xk = xk1
            break

        xk = xk1
        gk = gk1
        pk = pk1

    return xk, count, np.array(plist),np.array(glist)

In [15]:
def gradxFunc_ND(x):
    val=(x[0]-0.5)**2+(0.5*x[0]+x[1]-0.5)**2+(0.25*x[0]+0.5*x[1]+x[2]-0.5)**2                     
    return sp.jv(1,val)*(2*(x[0]-0.5)+(0.5*x[0]+x[1]-0.5)+0.5*(0.25*x[0]+0.5*x[1]+x[2]-0.5)) 

In [16]:
def gradyFunc_ND(x):
    val=(x[0]-0.5)**2+(0.5*x[0]+x[1]-0.5)**2+(0.25*x[0]+0.5*x[1]+x[2]-0.5)**2                                         
    return sp.jv(1,val)*((x[0]+2*x[1]-1)+0.25*x[0]+0.5*x[1]+x[2]-0.5) 

In [17]:
def gradzFunc_ND(x):
    val=(x[0]-0.5)**2+(0.5*x[0]+x[1]-0.5)**2+(0.25*x[0]+0.5*x[1]+x[2]-0.5)**2                                         
    return sp.jv(1,val)*(0.5*x[0]+x[1]+2*x[2]-1)

In [18]:
def grad_vec(x):
    grad=np.array([gradxFunc_ND(x),gradyFunc_ND(x),gradzFunc_ND(x)])
    return grad

In [35]:
start=[0,0,0]
ans1=conjugate_gradient(func_ND,grad_vec,start,500,1e-10)

In [36]:
ans1

(array([0.4969776 , 0.25535728, 0.24604867]),
 500,
 array([[ 6.11176304e-01,  5.23865403e-01,  3.49243602e-01],
        [ 1.60104276e-02,  8.75843990e-03,  7.12612610e-03],
        [ 1.90555481e-02,  9.92642971e-03,  8.31452882e-03],
        ...,
        [ 6.19875384e-06, -1.03966515e-05,  7.29450682e-06],
        [ 6.17279110e-06, -1.03569707e-05,  7.26782422e-06],
        [ 6.14699231e-06, -1.03175148e-05,  7.24128710e-06]]),
 array([[-6.11176304e-01, -5.23865403e-01, -3.49243602e-01],
        [-1.57184004e-02, -8.50813084e-03, -6.95925339e-03],
        [-1.12029428e-02, -5.63069364e-03, -4.81939074e-03],
        ...,
        [-4.55897058e-08,  8.03336836e-08, -5.75210156e-08],
        [-4.53168855e-08,  7.98705490e-08, -5.71971092e-08],
        [-4.50462711e-08,  7.94109400e-08, -5.68756397e-08]]))

In [37]:
lasth=ans1[2][-1,:]
lastg=ans1[3][-1,:]

In [38]:
np.dot(lasth,lastg)

-1.5080754666307412e-12

In [39]:
ans1[0]# Fletcher-Reeves

array([0.4969776 , 0.25535728, 0.24604867])

In [24]:
ans1[0]#Polak and Ribiere


array([0.46998097, 0.28500678, 0.23331086])


 #### Fletcher-reeves is more accurate compared to Polak-Ribiere

### $Gradient$ $Descent$

In [25]:
learning_rate=3
pos=np.array([0,0,0],dtype=np.float64)
for i in range(1000):
    prev=func_ND(pos)                                                        
    delx=gradxFunc_ND(pos)
    dely=gradyFunc_ND(pos)
    delz=gradzFunc_ND(pos)
    pos[0]-=learning_rate*delx
    pos[1]-=learning_rate*dely
    pos[2]-=learning_rate*delz
    print (pos, func_ND(pos),i+1)
    if abs(prev-func_ND(pos))<1e-10:
        break
    else:
        print (abs(prev-func_ND(pos)))

[1.83352891 1.57159621 1.04773081] 1.2845434652665406 1
0.2683078936311718
[-2.35758622 -2.78354228 -1.65794379] 1.4150170133850573 2
0.13047354811851664
[-0.45678944 -0.75598725 -0.37312809] 1.2027129617508001 3
0.21230405163425714
[-4.44294889 -4.98974203 -3.0405556 ] 1.5480876430420474 4
0.34537468129124727
[-1.00493844 -1.32463326 -0.71950144] 1.577711199589674 5
0.029623556547626606
[-4.89788765 -5.45088074 -3.31303884] 1.4866427653041245 6
0.09106843428554945
[-1.11668622 -1.4262746  -0.76910453] 1.3561154624444005 7
0.13052730285972403
[1.2810071  1.10513813 0.81431372] 1.7149602647717688 8
0.35884480232736826
[ 0.18253102 -0.08286217  0.04750786] 0.6485114861454012 9
1.0664487786263677
[1.24336374 1.04143449 0.75168835] 1.8519312989246406 10
1.2034198127792393
[-1.71625316 -2.12035665 -1.25741569] 1.376818702948539 11
0.47511259597610156
[2.61972478 2.52000531 1.69558717] 1.632564131382178 12
0.2557454284336389
[5.23921446 5.32485685 3.48152794] 1.4532861283544733 13
0.17927800

[0.49113047 0.24049362 0.24394063] 0.5000000966136721 176
1.1195983029566037e-09
[0.49115583 0.24052079 0.24395794] 0.5000000955132271 177
1.1004449573803754e-09
[0.49118099 0.24054773 0.2439751 ] 0.5000000944315008 178
1.0817262641182879e-09
[0.49120593 0.24057445 0.24399212] 0.5000000933680705 179
1.0634303437839776e-09
[0.49123065 0.24060093 0.24400899] 0.5000000923225256 180
1.0455448729018713e-09
[0.49125517 0.2406272  0.24402572] 0.5000000912944667 181
1.0280589712863275e-09
[0.49127949 0.24065324 0.24404231] 0.5000000902835059 182
1.0109607595509829e-09
[0.4913036  0.24067907 0.24405876] 0.5000000892892655 183
9.942403567109181e-10
[0.49132751 0.24070468 0.24407508] 0.5000000883113787 184
9.77886882580492e-10
[0.49135123 0.24073009 0.24409126] 0.5000000873494884 185
9.618902341301805e-10
[0.49137475 0.24075528 0.24410731] 0.5000000864032471 186
9.462413075311815e-10
[0.49139808 0.24078027 0.24412323] 0.5000000854723172 187
9.309298887316686e-10
[0.49142123 0.24080506 0.24413902]

[0.49382288 0.24337769 0.2457778 ] 0.5000000227354899 361
1.2699874485377904e-10
[0.49383145 0.24338688 0.24578365] 0.500000022609547 362
1.2594292275736052e-10
[0.49383999 0.24339602 0.24578947] 0.5000000224846481 363
1.2489886902500302e-10
[0.49384849 0.24340513 0.24579528] 0.5000000223607819 364
1.2386625058979917e-10
[0.49385696 0.2434142  0.24580106] 0.500000022237937 365
1.2284484540714402e-10
[0.4938654  0.24342324 0.24580681] 0.5000000221161022 366
1.2183487552164252e-10
[0.49387379 0.24343224 0.24581254] 0.5000000219952663 367
1.208358968440848e-10
[0.49388216 0.2434412  0.24581825] 0.5000000218754185 368
1.198477983521684e-10
[0.49389049 0.24345012 0.24582393] 0.5000000217565481 369
1.1887035800128842e-10
[0.49389878 0.24345901 0.2458296 ] 0.5000000216386445 370
1.1790357579144484e-10
[0.49390705 0.24346786 0.24583523] 0.5000000215216972 371
1.169473407003352e-10
[0.49391527 0.24347667 0.24584085] 0.5000000214056959 372
1.160013196610521e-10
[0.49392347 0.24348545 0.24584644]

In [26]:
import torch
from torch.autograd.functional import hessian

In [27]:
import scipy

In [28]:
import numdifftools

In [29]:
hessian_mat=numdifftools.Hessian(func_ND)([0.5, 0.25,0.25])

In [30]:
eigen_val,eigen_vec=eigenvalues_eigenvector(hessian_mat)

In [31]:
eigen_val

array([ 7.71146662e-12, -5.30142979e-13, -1.14448811e-11])

In [32]:
eigen_vec

array([[ 0.63649123, -0.19220055,  0.74695238],
       [-0.60449459,  0.4771708 ,  0.63788269],
       [ 0.47902526,  0.85753541, -0.18753084]])

In [33]:
#conjugate directions from conjugate gradient descent

ans1[2][-1,:]

array([ 2.91226534e-05, -2.23551962e-05,  8.98194231e-06])